In [1]:
import excelify as el
import numpy as np

### Scratch

In [3]:
a_df = el.ExcelFrame({"x": [1, 2, 3]})
a_df = a_df.with_columns(el.ConstantExpr(1).alias("y"))
a_df = a_df.with_columns((el.col("x") + el.col("y")).alias("z"))
a_df = a_df.with_columns(el.sum("x").alias("x_sum"))

b_df = el.ExcelFrame({"x": [4, 5, 6]})
b_df = b_df.with_columns((el.col("x") + el.col("x", from_=a_df)).alias("y"))
b_df = b_df.with_columns((el.col("x") + el.col("y")).alias("z"))
b_df = b_df.with_columns(el.sum("x").alias("x_sum"))
c_df = el.concat([a_df, b_df])
c_df

x,y,z,x_sum
1.0,1.0,(A1 + B1),SUM(A1:A3)
2.0,1.0,(A2 + B2),SUM(A1:A3)
3.0,1.0,(A3 + B3),SUM(A1:A3)
4.0,(A4 + A1),(A4 + B4),SUM(A4:A6)
5.0,(A5 + A2),(A5 + B5),SUM(A4:A6)
6.0,(A6 + A3),(A6 + B6),SUM(A4:A6)


In [3]:
c_df.to_excel("example.xlsx")

In [5]:
a_df = el.ExcelFrame({"x": [1, 2, 3], "y": [4, 5, 6]})
b_df = el.ExcelFrame({"a": [7, 8, 9]})
b_df = b_df.with_columns(
    (el.col("x", from_=a_df) + el.col("a")).alias("a + x")
)
b_df = b_df.select(["a + x", "a"])
b_df.evaluate()

a + x,a
8.0,7.0
10.0,8.0
12.0,9.0


### Design

Programmable Excel - Maybe a better VBA

Why is this different from dataframes?
Dataframe is not a DAG computation graph, but Excel is. But Excel looks very similar to Dataframe in a way that it's tabular.

So I'd like to build a python library that makes it easier to define an excel-like table, like:

```python

df = el.excelFrame({"x": [1, 2, 3]})
df = df.with_columns(
    # It'll define "y" as "x" column cell * 2 for each row.
    (el.col("x") * 2).alias("y"),
    # This is a common financial excel pattern (e.g. previous earnings * expected growth rate).
    (el.col("y").prev(1) * el.col("x")).alias("z"),
    # Empty column.
    (el.empty()).alias("empty_col")
)

df["empty_col"][2] = df["x"][0] + df["y"][0]
df["y"][:3].clear()

df.write_excel()

```

## DCF using Python

### Step 1: Raw Input Data

In [53]:
import excelify as el
import numpy as np

In [54]:
start_year = 2019
tax_rates = [6858. / 20564., 4915. / 20116., 4281. / 11460.]
effective_tax_rate = sum(tax_rates) / len(tax_rates)

In [55]:
columns = [
    "Year",
    "Retail Square Foot",
    "Net Sales",
    "Membership & Other Income",
    "Operating Income (EBIT)",
    "Capital Expenditures",
    "Depreciation & Amortization",
    "Sales per Square Foot",
    "COGS and OpEx per Square Foot",
    "Maintenance CapEx per Square Foot",
    "D&A per Square Foot",
    "Total Revenue",
    "(-) Taxes, Excluding Effect of Interest",
    "Net Operating Profit After Tax (NOPAT)",
]

[
    year,
    retail_sqft,
    net_sales,
    other_income,
    ebit,
    capex,
    d_and_a,
    sales_per_sqft,
    cogs_and_opex_per_sqft,
    maintenance_capex_per_sqrt,
    d_and_a_per_sqft,
    total_revenue,
    taxes,
    nopat,
] = [el.col(c) for c in columns]

In [56]:
df = el.ExcelFrame(
    {
        "Year": [f"FY{(start_year + i) % 100}" for i in range(3)],
        "Retail Square Foot": [1129, 1129, 1121],
        "Net Sales": [510_329, 519_926, 555_233],
        "Membership & Other Income": [4076, 4038, 3918],
        "Operating Income (EBIT)": [21_957, 20_568, 22_548],
        "Capital Expenditures": [-10_344, -10_705, -10_264],
        "Depreciation & Amortization": [10_678, 10_987, 11_152],
    }
)
fy18_retail_square_foot = 1158

In [36]:
df

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization
FY19,1129,510329,4076,21957,-10344,10678
FY20,1129,519926,4038,20568,-10705,10987
FY21,1121,555233,3918,22548,-10264,11152


In [57]:
[quarter, retail_square_foot, net_sales, other_income, ebit, capex, d_and_a] = [
    el.col(c) for c in df.columns
]

In [58]:
df = df.with_columns(
    (net_sales / retail_square_foot).alias("Sales per Square Foot"),
    ((net_sales - ebit) / retail_square_foot).alias("COGS and OpEx per Square Foot"),
    (-capex / retail_square_foot.prev(1)).alias("Maintenance CapEx per Square Foot"),
    (d_and_a / retail_square_foot).alias("D&A per Square Foot"),
    (net_sales + other_income).alias("Total Revenue"),
    (-ebit * effective_tax_rate).alias("(-) Taxes, Excluding Effect of Interest"),
)
df = df.with_columns(
    (el.col("(-) Taxes, Excluding Effect of Interest") + ebit).alias(
        "Net Operating Profit After Tax (NOPAT)"
    ),
)
df["Maintenance CapEx per Square Foot"][0] = -df["Capital Expenditures"][
    0
] / el.Constant(fy18_retail_square_foot)

In [59]:
df.evaluate()

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization,Sales per Square Foot,COGS and OpEx per Square Foot,Maintenance CapEx per Square Foot,D&A per Square Foot,Total Revenue,"(-) Taxes, Excluding Effect of Interest",Net Operating Profit After Tax (NOPAT)
FY19,1129,510329,4076,21957,-10344,10678,452.0186005314438,432.5704162976085,8.932642487046632,9.45792736935341,514405,-6963.212487758894,14993.787512241106
FY20,1129,519926,4038,20568,-10705,10987,460.51904340124,442.30115146147034,9.481842338352525,9.731620903454385,523964,-6522.719608699956,14045.280391300044
FY21,1121,555233,3918,22548,-10264,11152,495.30151650312223,475.18733273862625,9.091231178033658,9.948260481712756,559151,-7150.6360237731715,15397.363976226829


### Step 2: Projections

TODO: Think about how to organize these user inputs more nicely.

In [60]:
num_projecting_years = 10

In [61]:
projected_years = [f"FY{22 + i}" for i in range(10)]
retail_square_foot_gr = [0.02] * 2 + [0.015] * 2 + [0.01] * 2 + [0.005] * 4
sales_per_sqft_gr = [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2
cogs_and_opex_per_sqft_gr = sales_per_sqft_gr
maintenance_capex_per_sqft_gr = sales_per_sqft_gr
d_and_a_per_sqft_gr = [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2 + [0.008] * 2
initial_growth_capex_per_sqft = 150.0
growth_capex_per_sqft_gr = (
    [np.nan] + [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01]
)
membersip_and_other_income_gr = (
    [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2
)

In [62]:
manual_inputs = el.ExcelFrame(
    {
        "Year": projected_years,
        "Retail Square Foot Growth Rate": retail_square_foot_gr,
        "Sales per Square Foot Growth Rate": sales_per_sqft_gr,
        "COGS and OpEx per Square Foot Growth Rate": cogs_and_opex_per_sqft_gr,
        "Maintenance CapEx per Square Foot Growth Rate": maintenance_capex_per_sqft_gr,
        "D&A per Square Foot Growth Rate": d_and_a_per_sqft_gr,
        "Growth CapEx per New Square Foot Growth Rate": growth_capex_per_sqft_gr,
        "Membership & Other Income Growth Rate": membersip_and_other_income_gr,
    }
)
manual_inputs

Year,Retail Square Foot Growth Rate,Sales per Square Foot Growth Rate,COGS and OpEx per Square Foot Growth Rate,Maintenance CapEx per Square Foot Growth Rate,D&A per Square Foot Growth Rate,Growth CapEx per New Square Foot Growth Rate,Membership & Other Income Growth Rate
FY22,0.02,0.03,0.03,0.03,0.025,nan,0.03
FY23,0.02,0.03,0.03,0.03,0.025,0.03,0.03
FY24,0.015,0.025,0.025,0.025,0.02,0.03,0.025
FY25,0.015,0.025,0.025,0.025,0.02,0.025,0.025
FY26,0.01,0.02,0.02,0.02,0.015,0.025,0.02
FY27,0.01,0.02,0.02,0.02,0.015,0.02,0.02
FY28,0.005,0.015,0.015,0.015,0.01,0.02,0.015
FY29,0.005,0.015,0.015,0.015,0.01,0.015,0.015
FY30,0.005,0.01,0.01,0.01,0.008,0.015,0.01
FY31,0.005,0.01,0.01,0.01,0.008,0.01,0.01


TODO: Currently, the ordering of the columns are a mess. Fix this.

In [66]:
projection_df = el.ExcelFrame(
    {
        col: [None for _ in range(num_projecting_years)]
        for col in df.columns + ["Growth CapEx per New Square Foot"]
    }
)

for i in range(num_projecting_years):
    projection_df["Year"][i] = f"FY{(start_year + df.height + i) % 100}"

cash_flows = [
    "Retail Square Foot",
    "Sales per Square Foot",
    "COGS and OpEx per Square Foot",
    "Maintenance CapEx per Square Foot",
    "D&A per Square Foot",
    "Growth CapEx per New Square Foot",
    "Membership & Other Income",
]

for col in cash_flows:
    growth_rate_col = f"{col} Growth Rate"
    if col == "Growth CapEx per New Square Foot":
        projection_df[col][0] = 150.0
    else:
        projection_df[col][0] = df[col][-1] * (1.0 + manual_inputs[growth_rate_col][0])
    # Ideally I'd like to use column expression, but I haven't figured out how to express
    # other df's cell in the column expression.
    for i in range(1, num_projecting_years):
        projection_df[col][i] = projection_df[col][i - 1] * (
            1.0 + manual_inputs[growth_rate_col][i]
        )

projection_df = projection_df.with_columns(
    (retail_square_foot * sales_per_sqft).alias("Net Sales"),
    (net_sales - cogs_and_opex_per_sqft * retail_square_foot).alias(
        "Operating Income (EBIT)"
    ),
    # TODO: Fix the initial value.
    (
        -maintenance_capex_per_sqrt * retail_square_foot.prev(1)
        - el.col("Growth CapEx per New Square Foot")
        * (retail_square_foot - retail_square_foot.prev(1))
    ).alias("Capital Expenditures"),
    (retail_square_foot * d_and_a_per_sqft).alias("Depreciation & Amortization"),
    (net_sales + el.col("Membership & Other Income")).alias("Total Revenue"),
    (-el.col("Operating Income (EBIT)") * effective_tax_rate).alias(
        "(-) Taxes, Excluding Effect of Interest"
    ),
)
projection_df = projection_df.with_columns(
    (
        el.col("Operating Income (EBIT)")
        + el.col("(-) Taxes, Excluding Effect of Interest")
    ).alias("Net Operating Profit After Tax (NOPAT)")
)

TODO: If it's referring to a nonexistent cell, make it to None.

TODO: Jupyterlabb display is not the right medium to show the result of this. We do need a UI for this separately.

In [67]:
projection_df.evaluate()

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization,Sales per Square Foot,COGS and OpEx per Square Foot,Maintenance CapEx per Square Foot,D&A per Square Foot,Total Revenue,"(-) Taxes, Excluding Effect of Interest",Net Operating Profit After Tax (NOPAT),Growth CapEx per New Square Foot
FY22,1143.42,583327.7898,4035.54,23688.928799999994,-171513.0,11659.416,510.1605619982159,489.44295272078506,9.363968113374668,10.196966993755574,587363.3298000001,-7512.458206576092,16176.470593423903,150.0
FY23,1166.2884000000001,612844.17596388,4156.6062,24887.588597279857,-14561.32467280072,12189.919427999997,525.4653788581624,504.12624130240863,9.644887156775908,10.451891168599461,617000.7821638801,-7892.5885918287995,16995.000005451056,154.5
FY24,1183.7827260000001,637587.7595684218,4260.521355,25892.424986895174,-14313.897578523141,12620.223583808398,538.6020133296164,516.7293973349688,9.886009335695306,10.660928991971451,641848.2809234217,-8211.251856223933,17681.17313067124,159.135
FY25,1201.53946689,663330.3653609967,4367.034388874999,26937.831645741127,-14891.821193256004,13065.717476316833,552.0670636628568,529.6476322683429,10.133159569087688,10.87414757181088,667697.3997498717,-8542.781149918994,18395.050495822135,163.113375
FY26,1213.5548615589,683362.9423948986,4454.375076652499,27751.354161442374,-14427.767335295292,13394.320270846201,563.1084049361139,540.2405849137098,10.335822760469442,11.037259785388043,687817.3174715511,-8800.773140646503,18950.58102079587,167.19120937499997
FY27,1225.690410174489,704000.5032552247,4543.462578185549,28589.44505711808,-14863.485908821212,13731.187425657981,574.3705730348362,551.045396611984,10.542539215678831,11.202818682168862,708543.9658334103,-9066.556489494074,19522.888567624002,170.53503356249996
FY28,1231.8188622253613,718133.3133580732,4611.6145168583325,29163.378166639828,-14181.735645484521,13937.841796414134,582.9861316303587,559.3110775611636,10.700677303914013,11.314846868990552,722744.9278749316,-9248.5676110207,19914.81055561913,173.94573423374996
FY29,1237.977956536488,732549.8396237363,4680.7887346112075,29748.83298333513,-14466.43398856762,14147.606315450163,591.730923604814,567.700743724581,10.861187463472723,11.427995337680457,737230.6283583476,-9434.232605811943,20314.600377523187,176.5549202472562
FY30,1244.1678463191702,743574.7147100734,4727.596621957319,30196.55291973427,-14689.618097580162,14332.091101803633,597.6482328408622,573.3777511618268,10.96979933810745,11.519419300381902,748302.3113320308,-9576.217806529396,20620.335113204874,179.20324405096503
FY31,1250.3886855507658,754765.51416646,4774.872588176892,30651.01104117639,-14910.696849948732,14518.981569771147,603.6247151692709,579.1115286734452,11.079497331488524,11.611574654784956,759540.3867546369,-9720.339884517702,20930.67115665869,180.99527649147467


TODO

1. Create a separate projection df.
```python
projection_df = el.ExcelFrame({
    col: [None for _ in range(num_projecting_years)]
    for col in df.columns
})
```
3. Define a formula for each column.